## Shampoo Sales

In [1]:
# importing the general libraries

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
dataset = pd.read_csv('shampoo-sales.csv')
dataset.head()

,Month,Sales
0,1-01,266.0
1,1-02,145.9
2,1-03,183.1
3,1-04,119.3
4,1-05,180.3


In [3]:
def parser(x):
    return pd.datetime.strptime('190'+x, '%Y-%m')

series = pd.read_csv('shampoo-sales.csv', header = 0, index_col = 0, squeeze = True, parse_dates =True, date_parser = parser)
series

C:\Users\HP\anaconda3\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
  


Month
1901-01-01    266.0
1901-02-01    145.9
1901-03-01    183.1
1901-04-01    119.3
1901-05-01    180.3
1901-06-01    168.5
1901-07-01    231.8
1901-08-01    224.5
1901-09-01    192.8
1901-10-01    122.9
1901-11-01    336.5
1901-12-01    185.9
1902-01-01    194.3
1902-02-01    149.5
1902-03-01    210.1
1902-04-01    273.3
1902-05-01    191.4
1902-06-01    287.0
1902-07-01    226.0
1902-08-01    303.6
1902-09-01    289.9
1902-10-01    421.6
1902-11-01    264.5
1902-12-01    342.3
1903-01-01    339.7
1903-02-01    440.4
1903-03-01    315.9
1903-04-01    439.3
1903-05-01    401.3
1903-06-01    437.4
1903-07-01    575.5
1903-08-01    407.6
1903-09-01    682.0
1903-10-01    475.3
1903-11-01    581.3
1903-12-01    646.9
Name: Sales, dtype: float64

In [4]:
# checking for the number of observations

series.size

36

In [5]:
# descriptive statistics

print(series.describe())

count     36.000000
mean     312.600000
std      148.937164
min      119.300000
25%      192.450000
50%      280.150000
75%      411.100000
max      682.000000
Name: Sales, dtype: float64


## Feature Engineering

In [6]:
# creating date time feature of the series

dataframe = pd.DataFrame()


dataframe['year'] = [series.index[i].year for i in range(len(series))]
dataframe['month'] = [series.index[i].month for i in range(len(series))]
dataframe['day'] = [series.index[i].day for i in range(len(series))]
dataframe['sale'] = [series[i] for i in range(len(series))]

dataframe

,year,month,day,sale
0,1901,1,1,266.0
1,1901,2,1,145.9
2,1901,3,1,183.1
3,1901,4,1,119.3
4,1901,5,1,180.3
5,1901,6,1,168.5
6,1901,7,1,231.8
7,1901,8,1,224.5
8,1901,9,1,192.8
9,1901,10,1,122.9


In [12]:
# creating a lag feature for daily temperature dataset

sales = pd.DataFrame(series.values)
dataframe = pd.concat([sales.shift(1), sales], axis=1)
dataframe.columns = ['s', 's+1']
print(dataframe.head(5))

       s    s+1
0    NaN  266.0
1  266.0  145.9
2  145.9  183.1
3  183.1  119.3
4  119.3  180.3


In [15]:
# creating lag features (using the last three values)

sales = pd.DataFrame(series.values)
dataframe = pd.concat([sales.shift(3), sales.shift(2), sales.shift(1), sales], axis = 1)

dataframe.columns = ['s-2', 's-1', 's', 's+1']
print(dataframe.head(5))

     s-2    s-1      s    s+1
0    NaN    NaN    NaN  266.0
1    NaN    NaN  266.0  145.9
2    NaN  266.0  145.9  183.1
3  266.0  145.9  183.1  119.3
4  145.9  183.1  119.3  180.3


In [ ]:
# adding summaries of values



In [ ]:
# upsampling the (monthly) series into days

upsampled = series.resample('D').mean()
upsampled.head(38)

In [ ]:
# upsampling to dialy interver with linear interpolator

upsampled = series.resample('D').mean()
interpolated = upsampled.interpolate(method = 'linear')
print(interpolated.head(5))

# plot the lnear interpolated series

interpolated.plot()
plt.show()

In [ ]:
# interpolating the series using the using polynomia (or spline)

upsampled = series.resample('D').mean()
interpolated = upsampled.interpolate(method = 'spline', order = 2)
print(interpolated.head(5))

# ploting the interpolated series
interpolated.plot()
plt.show()

In [ ]:
# Down scaling the series into quarter

quarterly_mean_sale = series.resample('Q').mean()
print(quarterly_mean_sale.head(5))

# visualizing this
quarterly_mean_sale.plot()
plt.show()

In [ ]:
# downsampling the series into a yearly data

total_yearly_sale = series.resample('A').sum()
print(total_yearly_sale.head())

# vizualizing the series by yearly total sales

total_yearly_sale.plot()
plt.show()